## 언어모델
    - 단어의 시퀀스를 보고 다음 단어에 확률을 할당하는 모델
    - W1,...,Wn-1가 주어졌을때, n번째 단어 Wn으로 무엇이 올지를 예측하는 확률 모델
    - *다음 단어 예측하기(An adorable little boy is spreading __)
  

## 통계적 언어 모델
    - 언어모델이란 단어 시퀀스에 대한 확률분포를 의미
    - 언어모델은 m개 단어가 주어졌을때 m개 단어 시퀀스가 나타날 확률

## 언어모델의 이점
    - 언어모델은 불확실성을 단어 시퀀스의 출현 확률로 정량화가 가능하다는 장점

# 신경망 언어모델(Neural Network Language Model)
    - 통계적 언어 모델의 단점을 개선한 것이 신경망 언어모델
    - n-gram 언어 모델은 충분한 데이터를 관측하지 못하면 언어를 정확히 모델링하지 못하는 희소문제(sparsity problem). 예를들어 훈련 코퍼스에 'boy is spreading smile'라는 단어 시퀀스가 존재하지 않으면 n-gram 언어모델에서 해당 단어 시퀀스의 확률은 0이 된다.
    
    -결국 언어 모델 또한 단어의 의미적 유사성을 학습할 수 있도록 설계한다면, 훈련 코퍼스에 없는 단어 시퀀스에 대한 예측이라도 유사한 단어가 사용된 단어 시퀀스를 참고하여 보다 정확한 예측이 가능

## RNN의 문제점
       - 1. 하나의 Weight에 입력을 계속 쌓다보니 입력이 길어질수록 이전 입력에 대한 정보가 소실되는 기울기 소실(Vanishing Gradient)문제
       - 2. 단어별로 입력과 출력을 순환하는 RNN구조는 문장 생성엔 적합할지언정 번역에 사용하기는 어렵다는 문제가 있다.
       - 3. 긴 의존 기간으로 인한 문제점(우리가 현재 시점의 뭔가를 얻기 위해서 멀지 않은 최근의 정보만 필요로 할 때도 있다. 예를들어 이전 단어들을 토대로 다음에 올 단어를 예측하는 언어 모델을 생각해보자. 만약 우리가 "the coluds are in the sky"에서의 마지막 단어를 맞추고 싶다면, 저 문장 말고는 더 볼 필요도 없다. 이경우처럼 필요한 정보를 얻기 위한 시간 격차가 크지 않다면, RNN도 지난 정보를 바탕으로 학습이 가능.

# Sequence to Sequence 문제
    - 여러개 단어를 합쳐 고정된 크기의 Weight를 Linear로 처리하는 방식은 유연성에 한계가 존재.

# LSTM
    - 긴 의존 기간을 필요로 하는 학습을 수행할 능력을 보유(LSTM은 긴 의존 기간의 문제를 피하기 위해 명시적으로 설계)

## LSTM Encoder

In [1]:
import tensorflow as tf

class Encoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, enc_units):
    super(Encoder, self).__init__()
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.lstm = tf.keras.layers.LSTM(enc_units) # return_sequences 매개변수를 기본값 False로 전달

  def call(self, x):
    print("입력 Shape:", x.shape)

    x = self.embedding(x)
    print("Embedding Layer를 거친 Shape:", x.shape)

    output = self.lstm(x)
    print("LSTM Layer의 Output Shape:", output.shape)

    return output


In [4]:
vocab_size = 30000
emb_size = 256
lstm_size = 512
batch_size = 1
sample_seq_len = 3

print("Vocab Size: {0}".format(vocab_size))
print("Embedidng Size: {0}".format(emb_size))
print("LSTM Size: {0}".format(lstm_size))
print("Batch Size: {0}".format(batch_size))
print("Sample Sequence Length: {0}\n".format(sample_seq_len))

Vocab Size: 30000
Embedidng Size: 256
LSTM Size: 512
Batch Size: 1
Sample Sequence Length: 3



In [5]:
encoder = Encoder(vocab_size, emb_size, lstm_size)
sample_input = tf.zeros((batch_size, sample_seq_len))

sample_output = encoder(sample_input)    # 컨텍스트 벡터로 사용할 인코더 LSTM의 최종 State값

입력 Shape: (1, 3)
Embedding Layer를 거친 Shape: (1, 3, 256)
LSTM Layer의 Output Shape: (1, 512)


In [7]:
# Encoder 구현에 사용된 변수들을 이어 사용함에 유의!

class Decoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, dec_units):
    super(Decoder, self).__init__()
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.lstm = tf.keras.layers.LSTM(dec_units,
                                     return_sequences=True) # return_sequences 매개변수를 True로 설정
    self.fc = tf.keras.layers.Dense(vocab_size)
    self.softmax = tf.keras.layers.Softmax(axis=-1)

  def call(self, x, context_v):  # 디코더의 입력 x와 인코더의 컨텍스트 벡터를 인자로 받는다. 
    print("입력 Shape:", x.shape)

    x = self.embedding(x)
    print("Embedding Layer를 거친 Shape:", x.shape)

    context_v = tf.repeat(tf.expand_dims(context_v, axis=1),
                          repeats=x.shape[1], axis=1)
    x = tf.concat([x, context_v], axis=-1)  # 컨텍스트 벡터를 concat 해준다
    print("Context Vector가 더해진 Shape:", x.shape)

    x = self.lstm(x)
    print("LSTM Layer의 Output Shape:", x.shape)

    output = self.fc(x)
    print("Decoder 최종 Output Shape:", output.shape)

    return self.softmax(output)

In [9]:
print("Vocab Size: {0}".format(vocab_size))
print("Embedidng Size: {0}".format(emb_size))
print("LSTM Size: {0}".format(lstm_size))
print("Batch Size: {0}".format(batch_size))
print("Sample Sequence Length: {0}\n".format(sample_seq_len))

Vocab Size: 30000
Embedidng Size: 256
LSTM Size: 512
Batch Size: 1
Sample Sequence Length: 3



In [10]:
decoder = Decoder(vocab_size, emb_size, lstm_size)
sample_input = tf.zeros((batch_size, sample_seq_len))
dec_output = decoder(sample_input, sample_output)  # Decoder.call(x, context_v) 을 호출

입력 Shape: (1, 3)
Embedding Layer를 거친 Shape: (1, 3, 256)
Context Vector가 더해진 Shape: (1, 3, 768)
LSTM Layer의 Output Shape: (1, 3, 512)
Decoder 최종 Output Shape: (1, 3, 30000)


# seq2seq
    - : 고정된 길이의 sequence를 입력받아, 입력 sequence에 알맞은 길이의 sequence를 출력해주는 모델
      : encoder part와 decoder part로 구성된 모형으로 고정된 차원의 입력을 받아, 입력 값에 대응하는 가변적 길이의 결과 값을 출력해주는 모형

# Encoder
    - encoder는 입력 sequence를 순차적으로 LSTM에 입력받아 sequence의 마지막단에서 상태를 추출.
    "I like apple."이라는 영어 문장을 "나는 사과를 좋아합니다."라는 국문 문장으로 번역하는 예제를 생각해보겠습니다. 단어 단위로 Tokenization한 뒤 순차적으로 "I", "like", "an","apple","."을 입력받는다. 특히 "."은 입력 sequence의 종료를 나타내며 이를 별도로 End of Sentence (Eos)로 표기

# Decoder
    - 첫 LSTM에서는 입력으로 EOS를 받는다. EOS를 받는 이유는 학습 후 모형을 운영할 때 시작값을 알수 없기 때문에 학습단계에 강제로 eos로 시작하도록 구성.

# Context
    -: 마지막에 fixed-length context vector에서 4개의 단어에 대한 정보를 모두 함축.
     문장의 길이가 길다면, context vector에서 충분히 많은 정보를 압축하기를 기대하기 어렵다.

# Attention
    - Attention은 주어진 쿼리(Query)에서 대해서, 키(Key)에 대한 값(Value)을 이용하여 Query의 답을 얻기 위해서 어떤 Key를 살펴봐야 되는지 계산해주는 함수. Query는 attention의 주체를 의미하고, Key는 Query에 대한 attention의 기여도를 계산할 대상, Value는 Attention 크기를 계산하기 위한 값.
    - Q, K로 얻어진 Weight를 이용하여, Value의 Weighted sum으로 Attention Value가 출력
    - Attention(Q,K,V) = Attention Value
     : 어텐션 함수는 주어진 '쿼리(Query)'에 대해서 모든 '키'와의 유사도를 구한다. 그리고 구해낸 이 유사도를 키와 맵핑되어있는 각각의 '값(Value)'에 반영. 유사도가 반영된 '값'을 모두 더해서 리턴

Q = Query : t시점의 디코더 셀에서의 은닉 상태
K = Keys : 모든 시점의 인코더 셀의 은닉 상태들
V = Values: 모든 시점의 인코더 셀의 은닉 상태들

# Bahdanau Attention
    - bahdanau는 Seq2Seq2의 컨텍스트 벡터가 고정된 길이로 정보를  압축하는 것이 손실을 야기한다고 주장
    (문장이 길어질수록 성능이 저하)
    - bahdanau Attention은 Query를 Decoder t-1시점의 hidden state, Key, Value를 

In [12]:
class BahdanauAttention(tf.keras.layers.Layer):
  def __init__(self, units):
    super(BahdanauAttention, self).__init__()
    self.W_decoder = tf.keras.layers.Dense(units)
    self.W_encoder = tf.keras.layers.Dense(units)
    self.W_combine = tf.keras.layers.Dense(1)

  def call(self, H_encoder, H_decoder):
    print("[ H_encoder ] Shape:", H_encoder.shape)

    H_encoder = self.W_encoder(H_encoder)
    print("[ W_encoder X H_encoder ] Shape:", H_encoder.shape)

    print("\n[ H_decoder ] Shape:", H_decoder.shape)
    H_decoder = tf.expand_dims(H_decoder, 1)
    H_decoder = self.W_decoder(H_decoder)
    
    print("[ W_decoder X H_decoder ] Shape:", H_decoder.shape)

    score = self.W_combine(tf.nn.tanh(H_decoder + H_encoder))
    print("[ Score_alignment ] Shape:", score.shape)
    
    attention_weights = tf.nn.softmax(score, axis=1)
    print("\n최종 Weight:\n", attention_weights.numpy())

    context_vector = attention_weights * H_decoder
    context_vector = tf.reduce_sum(context_vector, axis=1)

    return context_vector, attention_weights

W_size = 100

print("Hidden State를 {0}차원으로 Mapping\n".format(W_size))

attention = BahdanauAttention(W_size)

enc_state = tf.random.uniform((1, 10, 512))
dec_state = tf.random.uniform((1, 512))

_ = attention(enc_state, dec_state)

Hidden State를 100차원으로 Mapping

[ H_encoder ] Shape: (1, 10, 512)
[ W_encoder X H_encoder ] Shape: (1, 10, 100)

[ H_decoder ] Shape: (1, 512)
[ W_decoder X H_decoder ] Shape: (1, 1, 100)
[ Score_alignment ] Shape: (1, 10, 1)

최종 Weight:
 [[[0.10010065]
  [0.12459187]
  [0.05294646]
  [0.07116408]
  [0.1209837 ]
  [0.09993205]
  [0.10715923]
  [0.09763453]
  [0.10073064]
  [0.12475681]]]


# Luong Attention 실습

In [13]:
class LuongAttention(tf.keras.layers.Layer):
  def __init__(self, units):
    super(LuongAttention, self).__init__()
    self.W_combine = tf.keras.layers.Dense(units)

  def call(self, H_encoder, H_decoder):
    print("[ H_encoder ] Shape:", H_encoder.shape)

    WH = self.W_combine(H_encoder)
    print("[ W_encoder X H_encoder ] Shape:", WH.shape)

    H_decoder = tf.expand_dims(H_decoder, 1)
    alignment = tf.matmul(WH, tf.transpose(H_decoder, [0, 2, 1]))
    print("[ Score_alignment ] Shape:", alignment.shape)

    attention_weights = tf.nn.softmax(alignment, axis=1)
    print("\n최종 Weight:\n", attention_weights.numpy())

    attention_weights = tf.squeeze(attention_weights, axis=-1)
    context_vector = tf.matmul(attention_weights, H_encoder)

    return context_vector, attention_weights

emb_dim = 512

attention = LuongAttention(emb_dim)

enc_state = tf.random.uniform((1, 10, emb_dim))
dec_state = tf.random.uniform((1, emb_dim))

_ = attention(enc_state, dec_state)

[ H_encoder ] Shape: (1, 10, 512)
[ W_encoder X H_encoder ] Shape: (1, 10, 512)
[ Score_alignment ] Shape: (1, 10, 1)

최종 Weight:
 [[[1.2928198e-04]
  [3.7399300e-03]
  [2.4067278e-05]
  [3.0471092e-01]
  [3.4939025e-05]
  [3.0040616e-02]
  [1.1312747e-01]
  [1.7242801e-06]
  [2.2881631e-02]
  [5.2530938e-01]]]
